# Drag Database V3

Satellite drag database to develop new machine learning algorithm that incoporates altitude.

## Datasets

All at 5 minute cadence

- Grace B
    - add geomagnetic coordinates
- Omni
- FISM2
- MSIS profiles

In [37]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [167]:
import os, sys
import pandas as pd

# for converting to
# geomagnetic coord
import aacgmv2

# add read_io module to current path ()
# and import
file_path = 'D:\\GitHub\\DataIO\\'
sys.path.append(os.path.dirname(file_path))
import data_io as dio

In [168]:
# dates to read in
sdate = '2002-01-01'
edate = '2016-01-01'

# number of years
ldate = pd.to_datetime(edate)-pd.to_datetime(sdate)  
ldate = int(ldate.total_seconds()/(365.2*86400)+1)

In [169]:
# load grace data

gr_d, gr_u, gr_m = dio.toleos_den.load_toleos(sat='gb',sdate=sdate,edate=edate)

d_min = gr_d['DateTime'].min()
d_max = gr_d['DateTime'].max()

NameError: name 'data_io' is not defined

In [164]:
# load omni data
om_d, om_m = dio.load_omni(res='5m',sdate=sdate, nd=ldate)

om_d = om_d[(om_d['DateTime'] >= d_min-pd.DateOffset(minutes=5)) & (om_d['DateTime'] <= d_max+pd.DateOffset(minutes=5))]

In [166]:
# read fsim2 data and truncate to similar range as grace data

fi_d, fi_m = dio.load_fism2()
fi_d = fi_d[(fi_d['DateTime'] >= d_min-pd.DateOffset(minutes=5)) & (fi_d['DateTime'] <= d_max+pd.DateOffset(minutes=5))]

,DateTime,Year,DOY,Hour,Minute,IMF_id,SW_id,IMF_pt,SW_pt,Per_int,...,SYM_D index,SYM_H index,ASY_D index,ASY_H index,PC index,Na_Np Ratio,MagnetosonicMach,Goes Proton flux (>10 MeV),Goes Proton flux (>30 MeV),Goes Proton flux (>60 MeV)
105115,2002-12-31 23:35:00,2002,365,23,35,60.0,60.0,5.0,5.0,100.0,...,-4.0,-16.0,15.0,12.0,0.09,4.2,0.11,0.04,0.03,NaN
105116,2002-12-31 23:40:00,2002,365,23,40,60.0,60.0,5.0,5.0,100.0,...,-4.0,-17.0,13.0,12.0,0.09,4.5,0.08,0.05,0.04,NaN
105117,2002-12-31 23:45:00,2002,365,23,45,60.0,60.0,4.0,4.0,100.0,...,-4.0,-16.0,13.0,11.0,0.13,4.3,0.05,0.03,0.02,NaN
105118,2002-12-31 23:50:00,2002,365,23,50,NaN,NaN,NaN,NaN,NaN,...,-4.0,-16.0,13.0,11.0,0.14,NaN,0.08,0.06,0.05,NaN
105119,2002-12-31 23:55:00,2002,365,23,55,NaN,NaN,NaN,NaN,NaN,...,-4.0,-16.0,13.0,11.0,0.19,NaN,0.07,0.05,0.04,NaN


In [152]:
om_d.iloc[0,[1, 2, 3, 4]]



dates = om_d.iloc[:,[1, 2, 3, 4]].astype('int32')

Year
DOY
Hour
Minute


In [ ]:
x = "f'"

for col in om_d.columns[[1, 2, 3, 4]]:
    
    x = x.join('''{x['col']:}''')

In [161]:
x = [1, 2, 3, 4]
len(x)

4

In [155]:
dates.dtypes

dt = [f"{x['Year']:04}{x['DOY']:03}{x['Hour']:02}{x['Minute']:02}" for index, x in dates.iterrows()]

In [159]:
dates.tail()

,Year,DOY,Hour,Minute
105115,2002,365,23,35
105116,2002,365,23,40
105117,2002,365,23,45
105118,2002,365,23,50
105119,2002,365,23,55


In [158]:
pd.to_datetime(dt,format='%Y%j%H%M')


DatetimeIndex(['2002-01-01 00:00:00', '2002-01-01 00:05:00',
               '2002-01-01 00:10:00', '2002-01-01 00:15:00',
               '2002-01-01 00:20:00', '2002-01-01 00:25:00',
               '2002-01-01 00:30:00', '2002-01-01 00:35:00',
               '2002-01-01 00:40:00', '2002-01-01 00:45:00',
               ...
               '2002-12-31 23:10:00', '2002-12-31 23:15:00',
               '2002-12-31 23:20:00', '2002-12-31 23:25:00',
               '2002-12-31 23:30:00', '2002-12-31 23:35:00',
               '2002-12-31 23:40:00', '2002-12-31 23:45:00',
               '2002-12-31 23:50:00', '2002-12-31 23:55:00'],
              dtype='datetime64[ns]', length=105120, freq=None)

In [115]:
fn = [f'{x.year}{x.DOY:03}{x.Hour:02}{x.Minute:02}' for x in dates.iterrows()]


AttributeError: 'tuple' object has no attribute 'year'

In [136]:
fn = [f"{int(x['DOY']):03}" for index, x in dates.iterrows()]

In [137]:
fn

['001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
